# Loading data

In [9]:
import os
import pickle

folder = "data_and_checkpoints"

with open(os.path.join(folder, "test.pkl"), "rb") as f:
    test_data = pickle.load(f)  

def process_data(test_data):
    modified_data = []
    for item in test_data:
        label_bio = item["label_BIO"].replace("Location-B", "location-B").replace("Location-I", "location-I")
        
        output = item["output"].replace("<Location>", "<location>").replace("</Location>", "</location>")
        
        modified_item = {
            "input": item["input"],
            "label_BIO": label_bio,
            "output": output,
            "label_json": item["label_json"]
        }
        modified_data.append(modified_item)
    return modified_data

# test
test_data = process_data(test_data)

# Experiments

In [10]:
import os
import csv
import copy
import asyncio
from tqdm.asyncio import tqdm_asyncio
import nest_asyncio
from prompt_teng import create_final_prompt, delresult, add_label_BIO, calculate_metrics
from langchain.chains.base import Chain
import time

# Experimental Parameter Configuration
MODEL_NAMES = [
    'deepseek-r1:7b','deepseek-r1:32b', 'deepseek-r1:671b',
]

# Result File Configuration
RESULTS_FILE = "model_experiment_results.csv"
ERROR_LOG = "model_experiment_errors.log"

def generate_parameters():
    """Generate all model parameters"""
    return [{'model_name': model} for model in MODEL_NAMES]

def load_completed_experiments():
    """Loading completed experiments"""
    completed = set()
    if os.path.exists(RESULTS_FILE):
        with open(RESULTS_FILE, 'r', newline='') as f:
            reader = csv.DictReader(f)
            for row in reader:
                completed.add(row['model_name'])
    return completed

async def async_invoke(chain: Chain, text: str, index: int, sem: asyncio.Semaphore): 
    """Asynchronous Call Model"""
    async with sem:
        try:
            answer = await chain.ainvoke({"input": text})
            return {"input": text, "output": answer.content.strip(), "index": index} 
        except Exception as e:
            return {"input": text, "output": None, "error": str(e), "index": index}

async def main_async(chain, text_list, concurrency=20):
    """Progressive asynchronous processing"""
    sem = asyncio.Semaphore(concurrency)
    # Establishing an index during task creation
    tasks = [async_invoke(chain, text, i, sem) for i, text in enumerate(text_list)]  
    results = []
    for future in tqdm_asyncio.as_completed(tasks, desc="Processing"):
        results.append(await future)
    # By indexing order
    return sorted(results, key=lambda x: x["index"]) 

def save_updated_data(updated_data, param):
    """Save the updated_data to a pkl file according to the model name."""
    os.makedirs(folder, exist_ok=True)
    model_name_safe = param['model_name'].replace(":", "_")
    filename = f"{model_name_safe}.pkl"
    filepath = os.path.join(folder, filename)
    try:
        with open(filepath, "wb") as f:
            pickle.dump(updated_data, f)
        print(f"Save as: {filepath}")
    except Exception as e:
        print(f"Save Failure: {str(e)}")

def run_single_experiment(param, test_data_copy):
    """Conducting a Single Model Experiment"""
    model_name = param['model_name']
    start = time.time() 
    # chain
    final_prompt, chain = create_final_prompt( model_name)
    text_list = [item['input'] for item in test_data_copy]

    # Executing Asynchronous Tasks
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    results = loop.run_until_complete(main_async(chain, text_list))
    
    delresult(results, test_data_copy, model_name)
    updated_data = add_label_BIO(test_data_copy, model_name, separator='\002')
    save_updated_data(updated_data, param)

    metrics = calculate_metrics(updated_data, model_name)
    metrics['duration_sec'] = round(time.time() - start, 2)
    return metrics

def main():
    parameters = generate_parameters()
    completed = load_completed_experiments()
    
    remaining_params = [p for p in parameters if p['model_name'] not in completed]
    
    print(f"Remaining experimental samples: {len(remaining_params)}")
    
    if not os.path.exists(RESULTS_FILE):
        with open(RESULTS_FILE, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['model_name', 'accuracy', 'precision', 'recall', 'f1', 'duration_sec'])
    
    for param in remaining_params:
        print(f"\n Ongoing experiment: {param['model_name']}")
        
        try:
            test_data_copy = copy.deepcopy(test_data)
            metrics = run_single_experiment(param, test_data_copy)
            
            with open(RESULTS_FILE, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    param['model_name'],
                    round(metrics['accuracy'], 4),
                    round(metrics['precision'], 4),
                    round(metrics['recall'], 4),
                    round(metrics['f1'], 4),
                    metrics['duration_sec']
                ])
            
            print(f"finshed: {param['model_name']}")
        
        except Exception as e:
            print(f"Error: {str(e)}")
            with open(ERROR_LOG, 'a') as f:
                f.write(f"{param['model_name']},{str(e)}\n")

if __name__ == "__main__":
    main()